<a href="https://colab.research.google.com/github/yojulab/study_AIs/blob/main/docs/MLs/SpineSurgeryList_GridSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터

In [1]:
import pandas as pd
df_SSL = pd.read_csv('SpineSurgeryList.csv')
df_SSL[:2]

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,1,2PT,44.9,4,26.0,0.0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0


In [4]:
df_SSL_extract = df_SSL[['성별','신장','체중', '흡연여부', '연령', '혈액형', '직업', '재발여부' ]]
# df_SSL_extract = df_SSL[['성별','신장','체중', '흡연여부', '연령', '재발여부' ]]
df_SSL_extract[:2]

,성별,신장,체중,흡연여부,연령,혈액형,직업,재발여부
0,2,163,60.3,0,66,RH+A,자영업,0
1,1,171,71.7,0,47,RH+A,운동선수,0


In [7]:
df_SSL_extract.isnull().sum()

성별      0
신장      0
체중      0
흡연여부    0
연령      0
혈액형     0
직업      0
재발여부    0
dtype: int64

### 전처리

In [6]:
df_SSL_extract.dropna(inplace=True)

<ipython-input-6-8f370ca9152c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SSL_extract.dropna(inplace=True)


In [8]:
target = df_SSL_extract['재발여부']
Xs = df_SSL_extract.drop(columns=['재발여부'])
features = pd.get_dummies(Xs)

In [9]:
features[:2]

,성별,신장,체중,흡연여부,연령,혈액형_RH+A,혈액형_RH+AB,혈액형_RH+B,혈액형_RH+O,직업_건설업,...,직업_사무직,직업_사업가,직업_예술가,직업_운동선수,직업_운수업,직업_의료직,직업_자영업,직업_주부,직업_특수전문직,직업_학생
0,2,163,60.3,0,66,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,171,71.7,0,47,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


## 특성공학 - Cross Validation

In [22]:
 # 모델:hyper parameters 교차 적용, 데이터셋 : 알맞게 섞어서 대입
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

hyper_parameters = {'max_depth':range(2, 10),
              'min_samples_leaf': range(2, 10),
              'criterion': ['gini','entropy'],
              'class_weight': [None, 'balanced'],
              'min_samples_split': range(2, 10)}

estimator_model = DecisionTreeClassifier()
# grid_model = GridSearchCV(estimator_model, hyper_parameters)

# score 방식 지정
from sklearn.metrics import f1_score, make_scorer
scoring = make_scorer(f1_score)
grid_model = GridSearchCV(estimator_model, hyper_parameters
                          , scoring=scoring
                          , cv=5)   # 데이터셋 폴드

grid_model

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 10),
                         'min_samples_leaf': range(2, 10),
                         'min_samples_split': range(2, 10)},
             scoring=make_scorer(f1_score))

## 모델

In [23]:
grid_model.fit(features, target)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 10),
                         'min_samples_leaf': range(2, 10),
                         'min_samples_split': range(2, 10)},
             scoring=make_scorer(f1_score))

In [26]:
grid_model.best_params_

{'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 4,
 'min_samples_leaf': 5,
 'min_samples_split': 2}

In [25]:
grid_model.scorer_

make_scorer(f1_score)

In [27]:
best_model = grid_model.best_estimator_

## 평가

In [28]:
from sklearn.metrics import classification_report
target_predict = best_model.predict(features)
print(classification_report(target, target_predict))

              precision    recall  f1-score   support

           0       0.93      0.79      0.85      1302
           1       0.27      0.58      0.37       177

    accuracy                           0.76      1479
   macro avg       0.60      0.68      0.61      1479
weighted avg       0.85      0.76      0.79      1479



In [29]:
target.value_counts()

0    1302
1     177
Name: 재발여부, dtype: int64